# FINN Instrumentation Wrapper Flow (Part 2/2)
#### **NOTE: Due to a bug with Vitis XSCT tools, the instrumentation wrapper will fail to run from this notebook. It must be run from outside the notebook (e.g. from the command line that the Jupyter notebook server was started from).**

**The code cells will not function as expected, and are only intended to be a guide for the commands that are used to run the instrumentation wrapper.** \
**The output products from the previous notebook (`1-build_model_and_platform.ipynb`) are required to run the instrumentation wrapper.** \
\
This Jupyter notebook will detail the process of taking the platform that was built in the previous notebook and through it, run the instrumentation wrapper on the hardware.\
\
In order the see the output, a serial terminal must be opened for the board that the instrumentation wrapper will run on. This could be done through commands like `screen <BOARD_SERIAL_PORT> <BAUD_RATE>`, or through programs like PuTTY. Additionally, it is assumed that the board is connected to a remote machine and a hw_server has been set up for it.

As detailed in the previous notebook, the parameters for your hw_server should be changed by editing the top-level `Makefile` in the `thin_platform` folder. This can be done by opening the file in the Jupyter notebook server and manually changing it, or by using the `sed` command, by replacing `(hw_server hostname)` with the hostname of your hardware server.

In [ ]:
#sed -i `s/<word_to_be_replaced>/<word_to_replace_with>/g` <file_to_change>

!sed -i 's/<HW_SERVER_HOST>/(hw_server hostname)/g' instr_wrap_platform/Makefile

Once this has been done, the instrumentation wrapper can be run using `make run_instr_wrap`. This will first call Vivado to source a `.tcl` script which export the `.xsa` hardware platform file for use in Vitis. It will then connect to the targeted device's hw_server and program the board with the generated `.pdi` device image file. Then, Vitis XSCT will be called to source a `.tcl` script which will build the platform and instrumentation wrapper application components, connect to the hw_server, and run the instrumentation wrapper application on the board. \
\
This `make` command may take a few minutes to run.

In [ ]:
#%%sh
#cd instr_wrap_platform
#make run_instr_wrap

The instrumentation wrapper output can then be seen on the serial terminal. The board will first be "booted up" by programming the `.pdi` file. Then, the instrumentation wrapper will begin polling the board as it runs the FINN design, and print out the various performance metrics. \
\
`status` checks to ensure that there is no timestamp overflow or underflow. The `interval` is the number of cycles it takes to read in and process one input and produce an output. The `latency` is the number of cycles between two produced outputs. The `checksum` is a verification value to check if the model is working as expected. \
\
The example built in these notebooks is relatively simple, so the latency and interval should be relatively low.

## Going further
These two notebooks provided a simple walkthrough of the build flow for the FINN instrumentation wrapper. The model used here was a relatively simple model which could be fully built in a few minutes. However, this build flow would work for any model which was built with the FINN compiler with the additional steps applied - the generated `.xo` files simply need to be copied to their respective locations in the Vitis IP directory, and then the `make` steps can be run to build and run the instrumentation wrapper.